# <center>Reconstrucción de Temperaturas Utilizando los *Boreholes* de Chile: Validación del Problema</center>

## 01 - Obtener las Anomalías en el Aire

Obtención de las anomalías en el aire para los *boreholes* ala1110-2 y 1501 en Chile desde un archivo netCDF.

In [3]:
#Importamos las librerías necesarias
from netCDF4 import Dataset as NetCDFFile

import pandas as pd
from datetime import timedelta,datetime

In [4]:
# Leer datos del archivos netCDF
data = NetCDFFile ("../datos_aire/gistemp250_GHCNv4.nc","r")
Float64 = "d"

In [5]:
# Latitude y longitude de los boreholes ala1110-2 y 1501 respectivamente
lat_borehole=[-28.66632,-26.75389]
lon_borehole=[-70.9104,-69.895]
nombres=["ala1110-2","1501"]

In [6]:
# Todas las latitudes dentro del netcdf
lat = data.variables["lat"][:]
lon = data.variables["lon"][:]

In [7]:
df_sat = pd.DataFrame()
for i in range(len(lat_borehole)):
    sq_diff_lat = (lat - lat_borehole[i])**2
    sq_diff_lon = (lon - lon_borehole[i])**2
    
    min_index_lat = sq_diff_lat.argmin()
    min_index_lon = sq_diff_lon.argmin()
    
    temp = data.variables["tempanomaly"]
    temp_borehole =  temp [:,min_index_lat,min_index_lon]
    
    #Store data a mi manera
    time_units = data.variables["time"].units
    starting_date = data.variables["time"].units[11:21]
    #print(starting_date) #just to know the number i have to add

    dates = []

    for data_indays in data.variables["time"]:
        specific_data = datetime(1800,1,1)
        new_date = str(specific_data + timedelta(int(data_indays)))
        new_date = new_date[0:9]
        dates.append(new_date)
        
    df_sat[f"{nombres[i]}"] = temp_borehole
df_sat["Fechas"] = dates  

In [8]:
# Reorganizar el dataframe, calculando la media anual
df_sat["Fechas"] = pd.to_datetime(df_sat.Fechas)
df_sat.set_index("Fechas",inplace=True)
df_sat = df_sat.resample("Y").mean()

# Guardamos el dataframe
df_sat.to_csv("..//resultados//sat.csv",sep=",")